In [1]:
import matplotlib.pyplot as plt
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.cluster import KMeans
import  seaborn as sns
import datasist as ds
import numpy as np


In [2]:
data = pd.read_csv('/home/xcrown/workshop/RL-trader/RL_trader_env/data/all_stocks_5yr.csv')
data.drop('Name',axis=1,inplace=True)

In [3]:
ds.structdata.describe(data)

First five data points


,Date,Open,High,Low,Close,Volume
0,2/8/2013,15.07,15.12,14.63,14.75,8407500
1,2/11/2013,14.89,15.01,14.26,14.46,8882000
2,2/12/2013,14.45,14.51,14.10,14.27,8126000
3,2/13/2013,14.30,14.94,14.25,14.66,10259500
4,2/14/2013,14.94,14.96,13.16,13.99,31879900




Random five data points


,Date,Open,High,Low,Close,Volume
597905,10/18/2016,63.05,63.15,62.45,62.54,941125
239628,4/19/2017,32.47,32.54,31.98,32.04,3214041
155296,1/10/2014,99.07,100.69,98.44,98.90,1210468
327972,12/23/2015,15.57,16.59,15.52,16.41,49429337
83102,7/27/2017,35.97,37.04,35.70,36.16,4730051




Last five data points


...

,count,unique,top,freq
Date,619040,1259,2/7/2018,505




Unique class Count of Categorical features


,Feature,Unique Count
0,Date,1259




Missing Values in Data


,features,missing_counts,missing_percent
0,Date,0,0.00
1,Open,11,0.00
2,High,8,0.00
3,Low,8,0.00
4,Close,0,0.00
5,Volume,0,0.00


In [4]:
# droping the rows with missing values of daily_return
data.dropna(inplace=True)

In [5]:
data.drop_duplicates(inplace=True)

In [6]:
# Calculate the IQR
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Remove outliers
df_out = data[~((data < (Q1 - 1.5 * IQR)) |(data > (Q3 + 1.5 * IQR))).any(axis=1)]
df_out.shape

(520354, 6)

In [7]:
from scipy.stats import zscore
# Calculate the z-scores
z_scores = zscore(data.drop('Date',axis=1))

# Define the threshold
threshold = 3

# Identify the outliers
outliers = np.where(np.abs(z_scores) > threshold)

dataset_no_outliers = data[(z_scores < threshold).all(axis=1)]

dataset_no_outliers.shape

(600847, 6)

The Local Outlier Factor (LOF) method, which is a density-based method for identifying outliers. The LOF algorithm calculates the local density deviation of a given data point with respect to its neighbors. A point with a high LOF score is considered an outlier, as it has a significantly lower density compared to its neighboring points. This method can be useful for detecting outliers in high-dimensional data.

In [10]:
from sklearn.neighbors import LocalOutlierFactor

# Creating an instance of the LocalOutlierFactor class
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

# Fiting the model to the data
lof.fit(data.drop('Date',axis=1))

# Geting the outlier scores for each data point
scores = lof.negative_outlier_factor_

# Identifing the outliers based on a threshold value
outliers_lof = data[scores < -threshold]

#  The Mahalanobis distance method

The Mahalanobis distance method, which takes into account the correlation between features. It calculates the distance of a given point from the mean of the data, but instead of using the standard Euclidean distance, it uses the Mahalanobis distance, which is a modified distance metric that takes into account the covariance structure of the data. Points that are far away from the mean using this distance metric are considered outliers. code for both

In [19]:
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.stats import chi2

# Generate some sample data
X = data.drop('Date',axis=1).to_numpy()
# Calculate the mean and covariance matrix of the data
mean = np.mean(X, axis=0)
cov = np.cov(X.T)

# Calculate the inverse of the covariance matrix
inv_cov = np.linalg.inv(cov)

# Calculate the Mahalanobis distance for each data point
distances = []
for x in X:
    d = mahalanobis(x, mean, inv_cov)
    distances.append(d)

# Calculate the threshold for outlier detection
alpha = 0.05
df = X.shape[1]
threshold = chi2.ppf(1 - alpha, df)

# Identify the outliers
outliers = np.where(distances > threshold)[0]


[   531   1259   1260 ... 604483 608337 615852]


I am creating a new feature in the dataset called 'daily_return'. The feature is calculated by taking the percentage change of the 'close' column. The percentage change, also known as the relative change, is a common method used to measure the change in a value over time. It is calculated by subtracting the previous value from the current value, and then dividing by the previous value. By taking the percentage change of the 'close' column, I am able to measure the daily return of the stock, which can be useful for understanding the volatility and performance of the stock over time. This feature can be useful for further analysis, such as identifying trends or patterns in the stock's performance, or for evaluating the risk and return of the stock relative to other stocks or benchmarks.